# Generative Adversarial Networks (GANs)

Neste notebook, exploraremos a teoria e a implementação de Redes Adversárias Generativas (GANs) aplicadas a um conjunto de dados bidimensional. As GANs são uma classe de modelos de aprendizado de máquina que consiste em duas redes neurais que competem entre si em um jogo de soma zero. Utilizaremos um dataset sintético para visualizar o processo de aprendizagem do gerador.

In [ ]:
import random
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Moons Dataset

### Preparando Dados

Para este estudo, utilizaremos o dataset `make_moons` da biblioteca `scikit-learn`. Este dataset é ideal para visualização, pois gera duas "luas" entrelaçadas em um espaço bidimensional. Ele nos permite observar diretamente como o gerador aprende a mapear uma distribuição de ruído para a distribuição complexa dos dados reais.

Vamos gerar os pontos, convertê-los para tensores do PyTorch e criar um `DataLoader` para iterar sobre os dados em lotes (`batches`) durante o treinamento.

In [ ]:
n_samples = 2000
noise = 0.05

X, y = make_moons(n_samples=n_samples, noise=noise, random_state=42)

plt.scatter(X[:, 0], X[:, 1], alpha=0.5)
plt.show()

### Redes Adversárias Generativas (GANs)

As GANs são compostas por duas redes neurais: o Gerador ($G$) e o Discriminador ($D$). Ambas são treinadas simultaneamente em um processo competitivo.

#### O Gerador e o Discriminador

* **Gerador ($G$)**: A sua função é aprender a criar dados sintéticos que sejam indistinguíveis dos dados reais. Ele recebe um vetor de ruído aleatório (vetor latente $z$), tipicamente amostrado de uma distribuição normal, e produz uma amostra de dados, $G(z)$, que se assemelha à distribuição dos dados de treinamento.

* **Discriminador ($D$)**: É uma rede classificadora binária que tenta distinguir entre amostras de dados reais e amostras falsas geradas pelo Gerador. Ele recebe uma amostra de dados ($x$) e retorna uma probabilidade, $D(x)$, que representa a chance de $x$ ser uma amostra real.

#### O Processo de Treinamento Adversário

O treinamento ocorre como um jogo. O Discriminador é treinado para maximizar sua acurácia ao classificar corretamente amostras reais e falsas. Simultaneamente, o Gerador é treinado para "enganar" o Discriminador, ou seja, para gerar amostras que o Discriminador classifique como reais.

#### A Função de Perda Minimax

O objetivo do treinamento da GAN é encontrar um equilíbrio de Nash no jogo entre o Gerador e o Discriminador. A função de valor $V(D, G)$ que o Discriminador tenta maximizar e o Gerador tenta minimizar é dada por:

$$
\min_{G} \max_{D} V(D, G) = \mathbb{E}_{x \sim p_{\text{data}}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_{z}(z)}[\log(1 - D(G(z)))]
$$

Nesta equação:
* $\mathbb{E}_{x \sim p_{\text{data}}(x)}$ é o valor esperado sobre as amostras reais $x$ da distribuição de dados $p_{\text{data}}$.
* $\mathbb{E}_{z \sim p_{z}(z)}$ é o valor esperado sobre os vetores de ruído $z$ da distribuição de ruído $p_{z}$.
* $D(x)$ é a probabilidade de que a amostra real $x$ seja real. O Discriminador quer maximizar este termo.
* $D(G(z))$ é a estimativa do Discriminador da probabilidade de que uma amostra falsa seja real. O Gerador quer maximizar este valor, enquanto o Discriminador quer minimizá-lo.

In [ ]:
real_data = torch.from_numpy(X).float()
dataset = TensorDataset(real_data)
batch_size = 64
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

### Implementação da GAN

Agora, implementaremos as redes Geradora e Discriminadora. Como nossos dados são bidimensionais, podemos usar arquiteturas de Perceptron de Múltiplas Camadas (MLP) relativamente simples.

#### Gerador

O Gerador receberá um vetor de ruído e o transformará em um ponto 2D. Utilizaremos camadas lineares e funções de ativação `ReLU`. A camada de saída não terá função de ativação, pois os dados não estão em um intervalo específico como `[-1, 1]`.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, output_dim),
        )

    def forward(self, z):
        return self.model(z)

#### Discriminador

O Discriminador receberá um ponto 2D e produzirá um único valor escalar indicando a probabilidade daquele ponto ser real. A camada de saída utilizará a função de ativação `Sigmoid` para mapear a saída para o intervalo `[0, 1]`.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 32),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)

### Treinamento do Modelo

O treinamento da GAN será realizado em um loop. Em cada época, iremos treinar o Discriminador e o Gerador de forma alternada.

In [ ]:
latent_dim = 16
output_dim = 2

G = Generator(latent_dim, output_dim).to(device)
D = Discriminator(output_dim).to(device)

In [ ]:
lr = 0.0002

# Função de perda
adversarial_loss = nn.BCELoss()

# Otimizadores
optimizer_G = torch.optim.Adam(G.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr)

#### O Loop de Treinamento

O loop de treinamento segue os passos canônicos da GAN:

1.  **Treinamento do Discriminador**: Atualizamos os pesos de $D$ para que ele maximize a probabilidade de classificar corretamente amostras reais (rótulo 1) e falsas (rótulo 0).
2.  **Treinamento do Gerador**: Atualizamos os pesos de $G$ para que ele maximize a probabilidade de $D$ classificar suas amostras geradas como reais (rótulo 1).

In [ ]:
n_epochs = 1000
plot_interval = 100

g_losses = []
d_losses = []

for epoch in range(n_epochs):
    for i, (real_samples,) in enumerate(train_dataloader):
        real_samples = real_samples.to(device)
        batch_size = real_samples.size(0)
        valid_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)

        # --- Treinamento do Discriminador ---
        optimizer_D.zero_grad()
        
        noise_vector = torch.randn(batch_size, latent_dim, device=device)
        generated_samples = G(noise_vector)
        
        real_loss = adversarial_loss(D(real_samples), valid_labels)
        fake_loss = adversarial_loss(D(generated_samples.detach()), fake_labels)
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()

        # --- Treinamento do Gerador ---
        optimizer_G.zero_grad()
        
        noise_vector = torch.randn(batch_size, latent_dim, device=device)
        generated_samples = G(noise_vector)
        
        g_loss = adversarial_loss(D(generated_samples), valid_labels)
        
        g_loss.backward()
        optimizer_G.step()
        
    g_losses.append(g_loss.item())
    d_losses.append(d_loss.item())
    
    if epoch % plot_interval == 0 or epoch in {1, 2, 5, 10}:
        print(f"[Epoch {epoch}/{n_epochs}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
        
        with torch.no_grad():
            z_plot = torch.randn(n_samples, latent_dim, device=device)
            generated_for_plot = G(z_plot).cpu().numpy()
        
        plt.figure(figsize=(6, 6))
        plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
        plt.scatter(generated_for_plot[:, 0], generated_for_plot[:, 1], alpha=0.7, c='red', s=10)
        plt.title(f"Epoch {epoch}")
        plt.show()

## Aplicação: MNIST

Após visualizar o comportamento da GAN em um ambiente bidimensional controlado, vamos agora aplicar o mesmo conceito a um problema mais complexo e prático: a geração de imagens de dígitos manuscritos a partir do dataset MNIST. A arquitetura das redes Geradora e Discriminadora continuará sendo baseada em Perceptrons de Múltiplas Camadas (MLPs), mas adaptada para lidar com as dimensões das imagens (28x28 pixels).

### Carregamento e Preparação dos Dados

Primeiramente, carregamos o dataset MNIST utilizando o `torchvision`. As imagens serão normalizadas para o intervalo `[-1, 1]`. Esta normalização é importante para que os valores dos pixels correspondam ao intervalo da função de ativação da camada de saída do Gerador, que será a Tangente Hiperbólica (`tanh`).

In [ ]:
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize

transform = Compose([
    ToTensor(),
    Normalize(mean=[0.5], std=[0.5])  # Normaliza os tensores para o intervalo [-1, 1]
])

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

validation_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

train_subset = Subset(training_data, range(20000))
validation_subset = Subset(validation_data, range(5000))

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)

for X, y in train_dataloader:
    print(f"X [N, C, H, W]: {X.shape}")
    print(f"y: {y.shape} {y.dtype}")
    break

### Definição dos Modelos

As arquiteturas do Gerador e do Discriminador serão MLPs. O Gerador receberá um vetor do espaço latente e sua tarefa será gerar uma imagem achatada de 784 pixels (28x28). O Discriminador, por sua vez, receberá uma imagem achatada e deverá produzir um valor de probabilidade indicando se a imagem é real ou falsa.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super().__init__()
        self.img_shape = img_shape
        output_dim = int(np.prod(img_shape))

        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(1024, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), *self.img_shape)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),

            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),

            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        x = img.view(img.size(0), -1)
        return self.model(x)

### Treinamento da GAN no MNIST

O processo de treinamento segue a mesma lógica adversária. Definimos os hiperparâmetros, instanciamos os modelos e otimizadores, e iniciamos o loop de treinamento, alternando entre a otimização do Discriminador e do Gerador.

In [ ]:
latent_dim = 100
img_shape = (1, 28, 28)

G = Generator(latent_dim, img_shape).to(device)
D = Discriminator(img_shape).to(device)

In [ ]:
# Função de perda
criterion = nn.BCELoss()

# Otimizadores
lr = 0.0002
G_optimizer = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_optimizer = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
n_epochs = 50
plot_interval = 5

g_losses = []
d_losses = []

for epoch in range(n_epochs):
    G.train(); D.train()
    g_running, d_running = 0, 0

    for imgs, _ in train_dataloader:
        valid = torch.ones(imgs.size(0), 1, device=device)
        fake = torch.zeros(imgs.size(0), 1, device=device)
        real_imgs = imgs.to(device)

        # --- Treina D ---
        D_optimizer.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = G(z).detach()
        real_loss = criterion(D(real_imgs), valid)
        fake_loss = criterion(D(gen_imgs), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        D_optimizer.step()

        # --- Treina G ---
        G_optimizer.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = G(z)
        g_loss = criterion(D(gen_imgs), valid)
        g_loss.backward()
        G_optimizer.step()

        d_running += d_loss.item()
        g_running += g_loss.item()

    g_losses.append(g_running / len(train_dataloader))
    d_losses.append(d_running / len(train_dataloader))

    if epoch % plot_interval == 0:
        print(f"[{epoch}/{n_epochs}] D: {d_losses[-1]:.4f} | G: {g_losses[-1]:.4f}")
        
        n_images = 8
        z = torch.randn(n_images, latent_dim, device=device)
        generated_imgs = G(z).detach().cpu()
        fig, axs = plt.subplots(2, 4, figsize=(8, 4))
        for i in range(n_images):
            row = i // 4
            col = i % 4
            axs[row, col].imshow(generated_imgs[i].squeeze(), cmap="gray")
            axs[row, col].axis("off")
        plt.show()

### Análise dos Resultados e Geração de Amostras

Após o treinamento, podemos analisar as curvas de perda. Idealmente, a perda do Discriminador ($D_{loss}$) deve permanecer em torno de 0.5, indicando que ele não consegue distinguir facilmente entre o real e o falso. A perda do Gerador ($G_{loss}$) deve diminuir, indicando que está melhorando em enganar o Discriminador.

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(g_losses, label="G")
plt.plot(d_losses, label="D")
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

#### Geração de Novos Dígitos

Agora, vamos usar o Gerador treinado para criar novos dígitos manuscritos. Para isso, basta passar um vetor de ruído aleatório pela rede do Gerador e visualizar a imagem resultante.

In [ ]:
n_images = 16
z = torch.randn(n_images, latent_dim, device=device)
generated_imgs = G(z).detach().cpu()
print(generated_imgs.shape)

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(8, 8))
for i in range(n_images):
    row = i // 4
    col = i % 4
    axs[row, col].imshow(generated_imgs[i].squeeze(), cmap="gray")
    axs[row, col].axis("off")
plt.show()

### Interpolação no Espaço Latente

Uma propriedade interessante das GANs é a capacidade de interpolar no espaço latente. O espaço latente é o espaço vetorial de onde os vetores de ruído $z$ são amostrados. Ao mover-se suavemente de um ponto a outro neste espaço, podemos gerar uma transição suave entre as imagens correspondentes.

Para demonstrar isso, vamos escolher dois vetores de ruído aleatórios, $z_1$ e $z_2$. Em seguida, criaremos uma sequência de vetores intermediários através de uma interpolação linear:

$$
z_{\text{interp}} = \alpha \cdot z_1 + (1 - \alpha) \cdot z_2
$$

onde $\alpha$ varia de 0 a 1. Passando cada $z_{\text{interp}}$ pelo Gerador, obteremos uma sequência de imagens que mostra uma transição de um dígito para outro.

In [ ]:
# Selecionar dois pontos aleatórios no espaço latente
z1 = torch.randn(1, latent_dim, device=device)
z2 = torch.randn(1, latent_dim, device=device)

# Número de passos para a interpolação
n_steps = 10
alpha_values = np.linspace(0, 1, n_steps)

# Gerar imagens interpoladas
interpolated_imgs = []
for alpha in alpha_values:
    # Interpolação linear
    z_interp = (1 - alpha) * z1 + alpha * z2
    
    # Gerar imagem
    img = G(z_interp).detach().cpu()
    interpolated_imgs.append(img)

In [ ]:
fig, axs = plt.subplots(1, n_steps, figsize=(15, 3))
for i in range(n_steps):
    axs[i].imshow(interpolated_imgs[i].squeeze(), cmap="gray")
    axs[i].axis("off")
    axs[i].set_title(f"α={alpha_values[i]:.2f}")
plt.show()